# 数据准备部分

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
def build_data(filename1,filename2):
    usage = pd.read_csv(filename1,header=None,usecols=[0,1,2,3])
#     print(usage.shape)
#     print(usage.head())
    
    usage.sort_values(by=[0],axis=0,inplace=True)
    usage.reset_index(inplace=True)
    
    usage.drop(labels=['index'],axis=1,inplace=True)
    
#     print(usage.shape)
#     print(usage.head())

        #异常值处理
    print('异常值处理！')
    usage = usage.values
    for i in tqdm(range(usage.shape[0])):
        if usage[i][2] > 21600:
            usage[i][2] = 21600
        if usage[i][2] < 5:
            usage[i][2] = 5
        if usage[i][3] > 50:
            usage[i][3] = 50
    usage = pd.DataFrame(usage)
    
    app_info = pd.read_csv(r'../data/app_info.csv',header=None)
#     print(app_info.shape)
#     print(app_info.head())

    usage_uId = (usage[0].value_counts())
    usage_uId = usage_uId.index
    
    row_index_dict = {}
    i = 0
    for user in usage.values:
        if user[0] not in row_index_dict.keys():
            row_index_dict[user[0]] = i
            i += 1
            
    app_category = {}
    for app_cate in app_info.values:
        app_category.setdefault(app_cate[0],[]).append(app_cate[1])
        
    app_category_appList = app_info.drop_duplicates(subset=[0])[0]
    usage_app_list = usage.drop_duplicates(subset=[1])[1]
    
    app_category_appList = set(list(set(app_category_appList).intersection(usage_app_list)))
    
    app_category_num = (app_info[1].value_counts()).index
    
    col_index_dict = {}
    i = 7
    for category in app_category_num:
        if category not in col_index_dict.keys():
            col_index_dict[category] = i
            i += 1
    print('数据存储！')      
    usage_on_category = []
    one_info = np.zeros(87)
    batch_size = 500
    Num,i =0, 0 
    flag = True
    preUserId = ''
    for user in tqdm(usage.values):
        currUserId = user[0]
        if flag==True:
            preUserId = currUserId
            flag = False
        if preUserId != currUserId:
            one_info[0] = preUserId
            usage_on_category.append(one_info)
            one_info, preUserId = np.zeros(87), currUserId
            i += 1
            if i==batch_size:
                usage_on_category = pd.DataFrame(usage_on_category)
                usage_on_category.to_csv(filename2,header=None,index=False,mode='a')
                usage_on_category, i = [], 0
        if user[1] not in app_category_appList:
            one_info[1] += user[2]
            one_info[2] += user[3]
            one_info[3] += 1
        else:
            one_info[4] += user[2]
            one_info[5] += user[3]
            one_info[6] += 1
            for category in app_category[user[1]]:
                col = col_index_dict[category]
                one_info[col] += user[2]
                one_info[col+40] += user[3]
    #尾部数据处理 
    one_info[0] = preUserId
    usage_on_category.append(one_info)
    usage_on_category = pd.DataFrame(usage_on_category)
    usage_on_category.to_csv(filename2,header=None,index=False,mode='a')
    return

In [3]:
filename1 = [r'../data/oneDayUsage/2019-03-04.csv',r'../data/oneDayUsage/2019-03-06.csv',r'../data/oneDayUsage/2019-03-02.csv']
filename2 = [r'../trainTestData/usageCategory/usage_on_category20190304.csv',r'../trainTestData/usageCategory/usage_on_category20190306.csv',r'../trainTestData/usageCategory/usage_on_category20190302.csv']

In [4]:
for i in [0,1,2]:
    build_data(filename1[i],filename2[i])

异常值处理！


  0%|          | 12830/22370609 [00:00<03:42, 100699.34it/s]

数据存储！


100%|██████████| 22370609/22370609 [03:16<00:00, 113786.51it/s]


异常值处理！


...

100%|██████████| 22312792/22312792 [03:15<00:00, 114412.58it/s]


异常值处理！


  0%|          | 6377/22848916 [00:00<08:01, 47458.73it/s]

数据存储！


100%|██████████| 22848916/22848916 [05:03<00:00, 75288.64it/s] 


# 生成三天的训练集和测试集（周一、周三、周六）

In [5]:
usage_monday = pd.read_csv(r'../trainTestData/usageCategory/usage_on_category20190304.csv',header=None)
usage_wednesday = pd.read_csv(r'../trainTestData/usageCategory/usage_on_category20190306.csv',header=None)
usage_saturday = pd.read_csv(r'../trainTestData/usageCategory/usage_on_category20190302.csv',header=None)

In [6]:
pd.set_option('max_columns',87)

In [7]:
print(usage_monday.shape)
print(usage_monday.head(3))

(1771311, 87)
          0     1    2    3        4      5     6       7       8       9   \
0  1000001.0   0.0  0.0  0.0   1436.0   83.0   3.0   556.0     5.0   551.0   
1  1000002.0   0.0  0.0  0.0  14262.0  354.0  15.0  5061.0  3953.0  3382.0   
2  1000003.0  11.0  4.0  2.0  18332.0  652.0  34.0  7618.0  3821.0  2814.0   

     10      11      12      13      14      15   16   17     18   19   20  \
0   0.0     0.0  1431.0   551.0     0.0     0.0  0.0  0.0    0.0  0.0  0.0   
1   0.0  1431.0  3762.0  8768.0  1756.0     0.0  0.0  0.0   34.0  0.0  0.0   
2  56.0   479.0  5127.0  7326.0   868.0  4156.0  0.0  0.0  343.0  0.0  0.0   

    21   22     23   24   25   26   27    28   29   30   31   32   33   34  \
0  0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.0    0.0  0.0  0.0  0.0  0.0  17.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0  233.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   

    35   36   37   38   39   40   41   42   43 

In [8]:
print(usage_wednesday.shape)
print(usage_wednesday.head(3))

(1763865, 87)
          0     1    2    3        4      5     6       7       8       9   \
0  1000001.0   0.0  0.0  0.0  14699.0  112.0   5.0  2532.0     5.0  2506.0   
1  1000002.0  28.0  1.0  1.0  10187.0  227.0  16.0  2693.0   789.0  1844.0   
2  1000003.0   0.0  0.0  0.0  15910.0  354.0  21.0  5556.0  9085.0  1899.0   

    10     11       12      13      14      15     16     17    18   19   20  \
0  0.0    0.0  14649.0  2482.0     0.0     0.0    0.0    0.0   0.0  0.0  0.0   
1  0.0  370.0   1796.0  3405.0  5445.0   319.0  140.0    0.0  43.0  0.0  0.0   
2  0.0   41.0   3371.0  3422.0  5934.0  2125.0    0.0  177.0   0.0  0.0  0.0   

    21   22   23   24   25   26   27    28   29   30   31   32   33   34   35  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  28.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

    36   37   38   39   40 

In [9]:
print(usage_saturday.shape)
print(usage_saturday.head(3))

(1785927, 87)
          0    1    2    3        4      5     6       7       8       9   \
0  1000001.0  0.0  0.0  0.0   6500.0  281.0  11.0  3753.0    77.0  2055.0   
1  1000002.0  0.0  0.0  0.0  34356.0  312.0  22.0  2615.0  1110.0  1948.0   
2  1000003.0  5.0  3.0  1.0  21679.0  522.0  23.0  7941.0  3471.0  2152.0   

      10      11      12      13      14      15   16      17    18   19   20  \
0    0.0     0.0  4724.0  2113.0     0.0     0.0  0.0     0.0   0.0  0.0  0.0   
1  105.0   100.0  2304.0  4675.0  5960.0  6628.0  0.0   133.0  70.0  0.0  0.0   
2   56.0  3612.0  5145.0  4399.0  3655.0  3194.0  0.0  1652.0   0.0  0.0  0.0   

    21   22       23   24   25     26   27    28   29   30   31   32   33  \
0  0.0  0.0    131.0  0.0  0.0  109.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.0  16142.0  0.0  0.0   84.0  0.0  50.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0      0.0  0.0  0.0    0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   

    34   35   36   37   38   39   40   41  

In [10]:
twoDay_usage = pd.merge(usage_monday,usage_wednesday,how='outer',left_on=0,right_on=0)
threeDay_usage = pd.merge(twoDay_usage,usage_saturday,how='outer',left_on=0,right_on=0)

In [11]:
del twoDay_usage,usage_monday, usage_wednesday, usage_saturday

In [12]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.impute import SimpleImputer

In [13]:
imp = SimpleImputer(strategy='constant',fill_value=0)

In [14]:
threeDay_usage = imp.fit_transform(threeDay_usage)

In [15]:
scale = MaxAbsScaler()

In [16]:
threeDay_usage[:,1:threeDay_usage.shape[1]] = scale.fit_transform(threeDay_usage[:,1:threeDay_usage.shape[1]])

In [17]:
threeDay_usage.shape

(1874425, 259)

In [18]:
age_train = pd.read_csv(r'../data/age_train.csv',header=None,usecols=[0])
age_test = pd.read_csv(r'../data/age_test.csv',header=None)

In [19]:
threeDay_usage = pd.DataFrame(threeDay_usage)

In [20]:
threeDay_usage.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258
0,1000001.0,0.000000,0.000000,0.000000,0.003754,0.017977,0.012146,0.003187,0.000054,0.008346,0.000000,0.000000,0.017668,0.005493,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.119914,0.010064,0.108411,0.000000,0.000000,0.095057,0.056133,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.048780,0.0,0.0,0.054945,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000002.0,0.000000,0.000000,0.000000,0.037280,0.076673,0.060729,0.029006,0.042671,0.051228,0.000000,0.022013,0.046448,0.087412,0.018917,0.000000,0.0,0.0,0.000619,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.057816,0.092406,0.093458,0.003770,0.023944,0.080798,0.113306,0.058151,0.037162,0.000000,0.037415,0.015190,0.0,0.0,0.000000,0.0,0.081301,0.0,0.0,0.016484,0.0,0.039146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000003.0,0.000121,0.001354,0.011765,0.047919,0.141217,0.137652,0.043661,0.041246,0.042624,0.000958,0.007369,0.063301,0.073036,0.009351,0.054122,0.0,0.0,0.006241,0.0,0.0,0.000000,0.0,0.004695,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111349,0.081427,0.093458,0.003231,0.070423,0.167300,0.108108,0.078170,0.047297,0.000000,0.132653,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000004.0,0.000000,0.000000,0.000000,0.041735,0.073641,0.068826,0.078857,0.042271,0.104909,0.000000,0.009984,0.094488,0.117649,0.000000,0.000000,0.0,0.0,0.050550,0.0,0.0,0.000955,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.074411,0.232388,0.093458,0.000000,0.160563,0.118821,0.141372,0.000000,0.000000,0.000000,0.000000,0.025316,0.0,0.0,0.041818,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000007.0,0.000055,0.000338,0.005882,0.044451,0.123673,0.101215,0.015801,0.021136,0.017692,0.014357,0.015722,0.017680,0.042629,0.112120,0.000000,0.0,0.0,0.002111,0.0,0.0,0.008073,0.0,0.000000,0.0,0.0,0.0,0.0,0.003047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.138651,0.175663,0.119626,0.057620,0.142254,0.123574,0.218295,0.050524,0.000000,0.007282,0.006803,0.017722,0.0,0.0,0.003636,0.0,0.030894,0.0,0.0,0.000000,0.0,0.213523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
usage_train = pd.merge(age_train,threeDay_usage,how='inner',left_on=0,right_on=0)
usage_train = pd.merge(age_train,usage_train,how='outer',left_on=0,right_on=0)

In [22]:
usage_test = pd.merge(age_test,threeDay_usage,how='inner',left_on=0,right_on=0)
usage_test = pd.merge(age_test,usage_test,how='outer',left_on=0,right_on=0)

In [23]:
del age_train, age_test, threeDay_usage

In [24]:
print(usage_train.shape)
print(usage_test.head())

(2010000, 259)
       0         1         2         3         4         5         6    \
0  1000002  0.000000  0.000000  0.000000  0.037280  0.076673  0.060729   
1  1000003  0.000121  0.001354  0.011765  0.047919  0.141217  0.137652   
2  1000004  0.000000  0.000000  0.000000  0.041735  0.073641  0.068826   
3  1000005       NaN       NaN       NaN       NaN       NaN       NaN   
4  1000006       NaN       NaN       NaN       NaN       NaN       NaN   

        7         8         9         10        11        12        13   \
0  0.029006  0.042671  0.051228  0.000000  0.022013  0.046448  0.087412   
1  0.043661  0.041246  0.042624  0.000958  0.007369  0.063301  0.073036   
2  0.078857  0.042271  0.104909  0.000000  0.009984  0.094488  0.117649   
3       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
4       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

        14        15   16   17        18   19   20        21   22        23   \
0  0.0189

In [25]:
print(usage_test.shape)
print(usage_test.head())

(502500, 259)
       0         1         2         3         4         5         6    \
0  1000002  0.000000  0.000000  0.000000  0.037280  0.076673  0.060729   
1  1000003  0.000121  0.001354  0.011765  0.047919  0.141217  0.137652   
2  1000004  0.000000  0.000000  0.000000  0.041735  0.073641  0.068826   
3  1000005       NaN       NaN       NaN       NaN       NaN       NaN   
4  1000006       NaN       NaN       NaN       NaN       NaN       NaN   

        7         8         9         10        11        12        13   \
0  0.029006  0.042671  0.051228  0.000000  0.022013  0.046448  0.087412   
1  0.043661  0.041246  0.042624  0.000958  0.007369  0.063301  0.073036   
2  0.078857  0.042271  0.104909  0.000000  0.009984  0.094488  0.117649   
3       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
4       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

        14        15   16   17        18   19   20        21   22        23   \
0  0.01891

In [26]:
usage_train = imp.fit_transform(usage_train.values)
usage_test = imp.fit_transform(usage_test.values)

In [27]:
usage_train = pd.DataFrame(usage_train[:,1:usage_train.shape[1]])
usage_test = pd.DataFrame(usage_test[:,1:usage_test.shape[1]])

In [28]:
print(usage_train.shape)
print(usage_train.head())

(2010000, 258)
   0    1    2         3         4         5         6         7         8    \
0  0.0  0.0  0.0  0.003754  0.017977  0.012146  0.003187  0.000054  0.008346   
1  0.0  0.0  0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.0  0.0  0.0  0.039115  0.021009  0.016194  0.046080  0.003228  0.120556   
3  0.0  0.0  0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.0  0.0  0.0  0.012552  0.014078  0.012146  0.011239  0.000000  0.029628   

   9         10        11        12   13   14   15   16   17   18   19   20   \
0  0.0  0.000000  0.017668  0.005493  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.000000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.003354  0.098267  0.079346  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3  0.0  0.000000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4  0.0  0.000000  0.024150  0.019500  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

   21        22   23   

In [29]:
print(usage_test.shape)
print(usage_test.head())

(502500, 258)
        0         1         2         3         4         5         6    \
0  0.000000  0.000000  0.000000  0.037280  0.076673  0.060729  0.029006   
1  0.000121  0.001354  0.011765  0.047919  0.141217  0.137652  0.043661   
2  0.000000  0.000000  0.000000  0.041735  0.073641  0.068826  0.078857   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

        7         8         9         10        11        12        13   \
0  0.042671  0.051228  0.000000  0.022013  0.046448  0.087412  0.018917   
1  0.041246  0.042624  0.000958  0.007369  0.063301  0.073036  0.009351   
2  0.042271  0.104909  0.000000  0.009984  0.094488  0.117649  0.000000   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

        14   15   16        17   18   19        20   21        22   23   24   \
0  0

In [30]:
from scipy import sparse

In [31]:
usage_train = sparse.csr_matrix(usage_train.values)
usage_test = sparse.csr_matrix(usage_test.values)

In [32]:
print(usage_train.shape)
print(usage_test.shape)

(2010000, 258)
(502500, 258)


In [33]:
sparse.save_npz(r'../trainTestData/trainData258.npz',usage_train)
sparse.save_npz(r'../trainTestData/testData258.npz',usage_test)